In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0


     |████████████████████████████████| 332.1MB 68kB/s 
     |████████████████████████████████| 419kB 49.6MB/s 
     |████████████████████████████████| 3.0MB 41.9MB/s 
     |████████████████████████████████| 61kB 30.8MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
from music21 import converter, instrument, note, chord, pitch
from google.colab import drive
import numpy as np
import time
import glob
import pickle


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = "/content/gdrive/My Drive/Applied Deep Learning/Data/"

In [0]:
embedding_size = 8
start_note = '<start>'
end_note = '<end>'


In [0]:
def get_notes(filename):
  notes = []
  midi = converter.parse(filename)
  notes_to_parse = None
  try:  # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except:  # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    new_chord = [-1] * 6 # 5 pitches + 1 duration
    
    if isinstance(element, note.Note):
      new_chord[0] = element.pitch.ps
      new_chord[5] = float(element.duration.quarterLength)
      notes.append(new_chord)
    elif isinstance(element, chord.Chord):
      new_chord[5] = float(element.duration.quarterLength)
      
      for k in range(len(element.pitches)):
        new_chord[k] = element.pitches[k].ps
        
        if k >= len(new_chord) - 1:
          break

      notes.append(new_chord)

  return notes


In [47]:
# pitch.Pitch(62)

<music21.pitch.Pitch D4>

In [0]:
test_notes = get_notes(data_path + '8.mid')

In [0]:
test_notes

In [81]:
np.zeros((2, 2)) - 1

array([[-1., -1.],
       [-1., -1.]])

In [84]:
# Parse abc to notesequence

from magenta.music import abc_parser as abc_parser
import numpy as np

MAX_NOTES = 5000
NUM_PARAM = 5 + 1 # 5 pitches + 1 duration

# 3D numpy array: num songs x MAX_NOTES x NUM_PARAM
all_songs_data = np.empty((1, MAX_NOTES, NUM_PARAM), np.float64)

for f in glob.glob(data_path + '*.mid'):
  song_notes = get_notes(f)

# for k in range(len(songs_wo_tuplets)):
#   res = abc_parser.parse_abc_tunebook(songs_wo_tuplets[k])[0]
  
#   for key in res.keys():
  temp_data = np.empty((1, 1, NUM_PARAM), np.float64)
  notes_data = np.zeros((1, MAX_NOTES, NUM_PARAM)) - 1
  
  for elem in song_notes:
    # Convert notes into a 2D numpy array
    # The 1st dimension is 5 pitches and 1 duration
    # The 2nd dimension is the number of notes
    raw_notes = np.array([[elem]]) 
    temp_data = np.append(temp_data, raw_notes, axis=1)

  notes_data[0, :temp_data.shape[1],:temp_data.shape[2]] = temp_data
#   print('notes_data = ', notes_data)
  
  all_songs_data = np.append(all_songs_data, notes_data, axis=0)

print(all_songs_data)


[[[ 7.26991432e-316  1.00000000e+000  1.00000000e+000  1.00000000e+000
    1.00000000e+000  1.66666667e+000]
  [-1.00000000e+000 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000 -1.00000000e+000]
  [-1.00000000e+000 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000 -1.00000000e+000]
  ...
  [-1.00000000e+000 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000 -1.00000000e+000]
  [-1.00000000e+000 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000 -1.00000000e+000]
  [-1.00000000e+000 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000 -1.00000000e+000]]

 [[ 6.90000000e+001  1.00000000e+000  1.00000000e+000  1.00000000e+000
    1.00000000e+000  1.33333333e+000]
  [ 5.20000000e+001  5.70000000e+001 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000  7.50000000e-001]
  [ 4.00000000e+001 -1.00000000e+000 -1.00000000e+000 -1.00000000e+000
   -1.00000000e+000  7.50000000e-001]
  ...
  [-1

In [85]:
all_songs_data.shape

(93, 5000, 6)

In [0]:
import pickle

pickle.dump( all_songs_data, open( "all_songs_data.p", "wb" ) )

all_songs_data = pickle.load( open( "all_songs_data.p", "rb" ) )

# Below is deprecated

In [0]:
songs = [' '.join([start_note] + get_notes(f) + [end_note]) for f in glob.glob(data_path + '*.mid')]
with open('songs_data', 'wb') as f:
    pickle.dump(songs, f)


In [0]:
print(songs[0])

<start> G5&0.25 F#5&0.25 D5&0.25 C5&0.25 B4&2.0 G2&0.5 D3&0.5 G3&1.0 0.4&2.0 11.2&2.0 G2&0.5 D3&0.5 G3&1.0 B4&0.25 C5&0.25 D5&0.25 F#4&0.25 G4&2.0 G2&0.5 D3&0.5 G3&1.0 9.0&2.0 7.11&3.0 G2&0.5 D3&0.5 G3&1.0 G2&0.5 D3&0.5 G5&0.5 G3&1.0 F#5&0.5 D5&1.5 E5&0.5 C5&0.5 G2&0.5 D5&0.5 D3&0.5 B4&2.0 G3&1.0 G5&0.25 F#5&0.25 D5&0.25 C5&0.25 B4&1.5 G2&0.5 D3&0.5 G3&1.0 C5&0.5 D5&1.0 F#4&1.0 G4&3.0 G2&0.5 D3&0.5 G3&1.0 G5&0.25 F#5&0.25 D5&0.25 C5&0.25 B4&1.0 G2&0.5 D3&0.5 G5&0.5 G3&1.0 F#5&0.5 D5&1.5 E5&0.5 C5&0.5 G2&0.5 D5&0.5 D3&0.5 B4&2.0 G3&1.0 B4&0.25 C5&0.25 D5&0.25 F#4&0.25 G4&1.5 G2&0.5 D3&0.5 G3&1.0 G4&0.5 A4&1.0 C5&1.0 B4&2.0 G2&0.5 D3&0.5 G3&1.0 E4&0.5 E4&0.5 F#4&0.5 G4&1.5 G3&2.5 E3&4.0 A4&0.5 B4&1.0 F#3&0.5 D5&1.0 G3&0.5 A3&0.5 F#4&3.0 B3&1.0 2.6&4.0 D4&3.0 F#4&0.5 D4&1.0 7.0&4.0 G4&0.5 C4&3.0 E4&2.0 E4&0.5 F#4&0.5 D4&3.0 2.7&4.0 C4&1.0 B3&2.0 G4&1.5 E3&4.0 G3&2.5 A4&0.5 B4&1.0 F#3&0.5 D5&0.5 G3&0.5 F#5&4.5 A3&0.5 2.6&4.0 B3&1.0 D4&3.0 F#5&0.5 0.4.7&2.0 G5&0.5 D5&0.5 C5&0.5 B4&0.5 5.9.0

In [0]:
with open('songs_data', 'rb') as f:
    songs = pickle.load(f)


In [0]:
# note_to_label = {(0, 0, 0):0}
# label_to_note = {0:(0, 0, 0)}

# data = np.load('Data.npy')
# tokenized = []
# for song in data:
#   s = [start_note]
#   for row in song:
#     note = tuple(row)
#     if note == (0, 0, 0):
#       continue
#     if note not in note_to_label:
#       note_to_label[note] = len(note_to_label) + 2
#       label_to_note[note_to_label[note]] = note
#     s.append(note_to_label[note])
#   s.append(end_note)
#   tokenized.append(s)

# num_notes = len(note_to_label) + 3


In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(songs)
tokenized = tokenizer.texts_to_sequences(songs)
num_notes = len(tokenizer.word_index) + 1


In [0]:
x = keras.preprocessing.sequence.pad_sequences(tokenized, padding='post')
n, maxlen = x.shape
y = np.hstack([x[:, 1:], np.zeros((n, 1), dtype=np.int)])

indexes = np.random.permutation(n)
split_index = int(0.9 * n)
x_train = x[indexes[:split_index]]
x_val = x[indexes[split_index:]]
y_train = y[indexes[:split_index]]
y_val = y[indexes[split_index:]]


In [0]:
# model = keras.models.Sequential()
# model.add(Embedding(notes_size, embedding_size))
# model.add(GRU(16, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(Dense(notes_size, activation='softmax'))
# model.summary()

class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.embedding = Embedding(num_notes, embedding_size)
    self.gru = GRU(16, input_shape=(2, 2212), \
                   return_sequences=True, return_state=True)
    self.dropout = Dropout(0.2)
    self.d = Dense(num_notes, activation='softmax')

  def call(self, x, state=None):
    x = self.embedding(x)
    x, output_state = self.gru(x, initial_state=state)
    x = self.dropout(x)
    x = self.d(x)
    return x, output_state

model = MyModel()


In [0]:
optimizer = keras.optimizers.Adam()
loss_metric_train = keras.metrics.Mean()
loss_metric_val = keras.metrics.Mean()

scc = keras.losses.SparseCategoricalCrossentropy()
def loss_function(y, output):
  return scc(y, output, sample_weight=tf.math.not_equal(y, 0))


In [0]:
@tf.function
def train_step(x, y, model):
  with tf.GradientTape() as tape:    
    output = model(x)[0]
    loss = loss_function(y, output)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  loss_metric_train(loss)


@tf.function
def val_step(x, y, model):
  output = model(x)[0]
  loss = loss_function(y, output)
  
  loss_metric_val(loss)


In [0]:
epochs = 10
batch_size = 1

# model.compile(optimizer='adam', loss=loss_function)
# model.fit(x, y, epochs=epochs, validation_split=0.1)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(x_val.shape[0])

start = time.time()
for epoch in range(epochs):
  for x, y in train_data:
    train_step(x, y, model)
  for x, y in val_data:
    val_step(x, y, model)
  if epoch % 1 == 0:
    end = time.time()
    print(f'Epoch {epoch}, Time: {end - start: .3f}s, Loss: {loss_metric_train.result(): .4f}, Validation Loss: {loss_metric_val.result(): .4f}')
    start = time.time()


Epoch 0, Time:  6.840s, Loss:  2.0733, Validation Loss:  3.1115
Epoch 1, Time:  4.409s, Loss:  1.9486, Validation Loss:  3.0247
Epoch 2, Time:  4.373s, Loss:  1.8741, Validation Loss:  2.9999
Epoch 3, Time:  4.235s, Loss:  1.8322, Validation Loss:  2.9914
Epoch 4, Time:  4.240s, Loss:  1.8055, Validation Loss:  2.9888
Epoch 5, Time:  4.226s, Loss:  1.7871, Validation Loss:  2.9887
Epoch 6, Time:  4.242s, Loss:  1.7736, Validation Loss:  2.9899
Epoch 7, Time:  4.246s, Loss:  1.7633, Validation Loss:  2.9915
Epoch 8, Time:  4.219s, Loss:  1.7552, Validation Loss:  2.9934
Epoch 9, Time:  4.227s, Loss:  1.7485, Validation Loss:  2.9943


In [0]:
def generate(model, k=10):
  result = [tokenizer.word_index[start_note]]
  state = None
  while True:
    seq = np.array([[result[-1]]])
    output, state = model(seq, state=state)
    note = np.random.choice(np.argpartition(output[0][0], -k)[-k:])
    if note == 0:
      result.append(tokenizer.word_index[end_note])
    else:
      result.append(note)
    if result[-1] == tokenizer.word_index[end_note] or len(result) >= 300:
      break
  return result[1:-1]


In [0]:
generated = generate(model)
print(len(generated))
print(generated)


298
[2, 6, 1, 3, 9, 9, 4, 1, 1, 8, 11, 1, 3, 7, 3, 8, 1, 9, 9, 11, 2, 5, 9, 6, 2, 9, 6, 7, 7, 3, 5, 9, 3, 7, 8, 6, 6, 7, 4, 2, 9, 5, 1, 9, 1, 5, 8, 1, 2, 11, 4, 5, 8, 3, 3, 5, 4, 5, 5, 9, 2, 3, 9, 7, 3, 4, 8, 2, 7, 1, 7, 7, 9, 4, 3, 5, 11, 2, 5, 2, 1, 3, 7, 2, 11, 1, 4, 2, 5, 7, 9, 4, 11, 8, 11, 2, 8, 11, 1, 11, 7, 2, 3, 6, 5, 5, 7, 5, 2, 4, 5, 8, 1, 5, 1, 4, 8, 11, 2, 2, 7, 4, 7, 3, 4, 1, 5, 2, 11, 3, 7, 9, 4, 4, 8, 11, 3, 2, 6, 1, 8, 11, 11, 11, 7, 9, 3, 6, 8, 2, 9, 8, 2, 11, 5, 4, 5, 1, 1, 4, 2, 4, 6, 11, 6, 9, 5, 11, 5, 11, 8, 2, 8, 4, 8, 4, 7, 7, 1, 2, 7, 3, 6, 8, 3, 8, 3, 4, 4, 7, 9, 3, 6, 3, 2, 6, 3, 8, 3, 3, 1, 6, 3, 1, 6, 11, 3, 7, 11, 5, 2, 3, 5, 1, 6, 11, 1, 2, 8, 6, 11, 7, 11, 4, 7, 4, 2, 4, 2, 9, 5, 2, 6, 6, 11, 6, 11, 8, 9, 2, 2, 6, 9, 11, 1, 3, 3, 11, 6, 2, 3, 6, 1, 9, 7, 8, 6, 11, 3, 6, 9, 4, 7, 1, 1, 9, 3, 4, 7, 3, 1, 7, 2, 1, 8, 3, 11, 2, 3, 11, 4, 5, 11, 6, 8, 2, 11, 2, 2, 11, 1, 6, 4, 3, 1, 7, 1, 7]


In [0]:
s = [tokenizer.index_word[i] for i in generated]
print(s)


['D3&0.5', 'C4&0.5', 'G3&0.5', 'C3&0.5', 'E4&0.5', 'E4&0.5', 'A3&0.5', 'G3&0.5', 'G3&0.5', 'E3&0.5', 'B3&0.5', 'G3&0.5', 'C3&0.5', 'D4&0.5', 'C3&0.5', 'E3&0.5', 'G3&0.5', 'E4&0.5', 'E4&0.5', 'B3&0.5', 'D3&0.5', 'A2&0.5', 'E4&0.5', 'C4&0.5', 'D3&0.5', 'E4&0.5', 'C4&0.5', 'D4&0.5', 'D4&0.5', 'C3&0.5', 'A2&0.5', 'E4&0.5', 'C3&0.5', 'D4&0.5', 'E3&0.5', 'C4&0.5', 'C4&0.5', 'D4&0.5', 'A3&0.5', 'D3&0.5', 'E4&0.5', 'A2&0.5', 'G3&0.5', 'E4&0.5', 'G3&0.5', 'A2&0.5', 'E3&0.5', 'G3&0.5', 'D3&0.5', 'B3&0.5', 'A3&0.5', 'A2&0.5', 'E3&0.5', 'C3&0.5', 'C3&0.5', 'A2&0.5', 'A3&0.5', 'A2&0.5', 'A2&0.5', 'E4&0.5', 'D3&0.5', 'C3&0.5', 'E4&0.5', 'D4&0.5', 'C3&0.5', 'A3&0.5', 'E3&0.5', 'D3&0.5', 'D4&0.5', 'G3&0.5', 'D4&0.5', 'D4&0.5', 'E4&0.5', 'A3&0.5', 'C3&0.5', 'A2&0.5', 'B3&0.5', 'D3&0.5', 'A2&0.5', 'D3&0.5', 'G3&0.5', 'C3&0.5', 'D4&0.5', 'D3&0.5', 'B3&0.5', 'G3&0.5', 'A3&0.5', 'D3&0.5', 'A2&0.5', 'D4&0.5', 'E4&0.5', 'A3&0.5', 'B3&0.5', 'E3&0.5', 'B3&0.5', 'D3&0.5', 'E3&0.5', 'B3&0.5', 'G3&0.5', 'B3&0.5',

In [0]:
generated_data = [generate(model) for i in range(2)]


In [0]:
new_songs = [[tokenizer.index_word[i] for i in g] for g in generated_data]


In [0]:
model.save_weights('my_model_weights.h5')


In [0]:
model.load_weights('my_model_weights.h5')


In [0]:
new_songs

[['B3&0.5',
  'A3&0.5',
  'D4&0.5',
  'B3&0.5',
  'C3&0.5',
  'A3&0.5',
  'D3&0.5',
  'C4&0.5',
  'A3&0.5',
  'E3&0.5',
  'D3&0.5',
  'E4&0.5',
  'A2&0.5',
  'E3&0.5',
  'G3&0.5',
  'C4&0.5',
  'G3&0.5',
  'B3&0.5',
  'C3&0.5',
  'G3&0.5',
  'C3&0.5',
  'D4&0.5',
  'E4&0.5',
  'A3&0.5',
  'B3&0.5',
  'C4&0.5',
  'E3&0.5',
  'A3&0.5',
  'E4&0.5',
  'C3&0.5',
  'D4&0.5',
  'D4&0.5',
  'A2&0.5',
  'G3&0.5',
  'A3&0.5',
  'E4&0.5',
  'C3&0.5',
  'B3&0.5',
  'D4&0.5',
  'C4&0.5',
  'G3&0.5',
  'E3&0.5',
  'D3&0.5',
  'D4&0.5',
  'A3&0.5',
  'D4&0.5',
  'A2&0.5',
  'E3&0.5',
  'B3&0.5',
  'E4&0.5',
  'A3&0.5',
  'C3&0.5',
  'A2&0.5',
  'G3&0.5',
  'E4&0.5',
  'C3&0.5',
  'D4&0.5',
  'E3&0.5',
  'C4&0.5',
  'A2&0.5',
  'D3&0.5',
  'D3&0.5',
  'G3&0.5',
  'C4&0.5',
  'D3&0.5',
  'A3&0.5',
  'D4&0.5',
  'E3&0.5',
  'A3&0.5',
  'D3&0.5',
  'C3&0.5',
  'D3&0.5',
  'A3&0.5',
  'B3&0.5',
  'E4&0.5',
  'D4&0.5',
  'G3&0.5',
  'A2&0.5',
  'A2&0.5',
  'C4&0.5',
  'E4&0.5',
  'A2&0.5',
  'G3&0.5',
  'A